<a href="https://colab.research.google.com/github/karumugamio/karumugamio.github.io/blob/master/NMT_Word_Level_Translation_Eng_to_Simple_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scikit-learn

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22.2.post1)
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
# Building a english to Tamil translator
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt

import re
from sklearn.model_selection import train_test_split

import os

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
print(pd.__version__)
%matplotlib inline


os.chdir('/gdrive/My Drive/NLAProjectWS')
os.listdir()

1.0.3


['Data', 'workspace', 'Readme.md', 'merged_dataset.txt']

Data Preparation for English to Tamil Translation

In [5]:
input_simple = '/gdrive/My Drive/NLAProjectWS/Data/v1_wiki.simple'
input_en = '/gdrive/My Drive/NLAProjectWS/Data/v1_wiki.unsimplified'

en_dataset=pd.read_csv(input_en,delimiter="\n",header=None,names = ['enSrc'])
simple_dataset = pd.read_csv(input_simple,delimiter="\n",header = None,names = ['simpleSrc'])

print(en_dataset.info())
print(simple_dataset.info())

### Print Head to Check the file
print(en_dataset.head())
print(simple_dataset.head())

src_df = en_dataset.copy()
src_df['simpleSrc'] = simple_dataset['simpleSrc']
print(src_df.info())
print(src_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137362 entries, 0 to 137361
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   enSrc   137362 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137362 entries, 0 to 137361
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   simpleSrc  137362 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB
None
                                               enSrc
0  When Green tore his ACL in a preseason game , ...
1  Royal Rumble was the twentieth annual Royal Ru...
2  Janko Prunk is a Slovenian historian of modern...
3  Sometimes the balalaika is tuned `` guitar sty...
4  Two days after he was sworn in as Prime Minist...
                                           simpleSrc
0  Warner was the backup quarterback for the St. ...
1  Royal Rumble was the twentieth y

Data Clean up 

In [0]:
src_df.enSrc=src_df.enSrc.apply(lambda x: x.lower())
src_df.simpleSrc=src_df.simpleSrc.astype(str)
src_df.simpleSrc=src_df.simpleSrc.apply(lambda x: x.lower())

In [0]:
# Take the length as 50
src_df.enSrc=src_df.enSrc.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
src_df.simpleSrc=src_df.simpleSrc.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [0]:
exclude = set(string.punctuation)
src_df.enSrc=src_df.enSrc.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
src_df.simpleSrc=src_df.simpleSrc.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
remove_digits = str.maketrans('', '', digits)
src_df.enSrc=src_df.enSrc.apply(lambda x: x.translate(remove_digits))
src_df.simpleSrc=src_df.simpleSrc.apply(lambda x: x.translate(remove_digits))

In [10]:
src_df.simpleSrc = src_df.simpleSrc.apply(lambda x : 'START_ '+ x + ' _END')
src_df.head()

,enSrc,simpleSrc
0,when green tore his acl in a preseason game C...,START_ warner was the backup quarterback for t...
1,royal rumble was the twentieth annual royal ru...,START_ royal rumble was the twentieth yearly r...
2,janko prunk is a slovenian historian of modern...,START_ janko prunk COMMA is a slovenian histo...
3,sometimes the balalaika is tuned guitar style...,START_ the balalaika family includes the prima...
4,two days after he was sworn in as prime minist...,START_ he became prime minister two days befor...


In [11]:
all_eng_words=set()
for eng in src_df.enSrc:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_simple_words=set()
for simpleEn in src_df.simpleSrc:
    for word in simpleEn.split():
        if word not in all_simple_words:
            all_simple_words.add(word)

print(len(all_eng_words), len(all_simple_words))

108470 98368


In [0]:
lenght_list=[]
for l in src_df.simpleSrc:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)

In [0]:
lenght_list=[]
for l in src_df.enSrc:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)

In [0]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_simple_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_simple_words)

In [15]:
print (num_encoder_tokens,num_decoder_tokens)

108470 98368


In [0]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [18]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.python.keras.models import Model
src_df = shuffle(src_df)
src_df.head(10)

,enSrc,simpleSrc
108328,itanagar is the capital of the indian state of...,START_ itanagar is the capital of the indian s...
3503,clottes concludes that the dates fall into tw...,START_ clottes concludes that the dates fall ...
61978,cowes and east cowes became a single urban dis...,START_ cowes is an english seaport town on the...
103168,then COMMA west held several posts in the adm...,START_ then COMMA west held several posts in ...
125934,the complex is located in travelcard zone,START_ the complex is located in travelcard zo...
96191,ashley is a city in washington county COMMA i...,START_ ashley is a city of illinois in the uni...
27152,the goldfields produced COMMA ounces of gold ...,START_ about COMMA ounces of gold were sent b...
55543,affectionately called dulce in northern beli...,START_ affectionately called dulce in northe...
88492,it is not clear how many slender lorises survi...,START_ it is not clear how many slender lorise...
95834,according to some studies COMMA the most freq...,START_ the type of incest most often reported ...


In [19]:
# Train - Test Split
X, y = src_df.enSrc, src_df.simpleSrc
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((123625,), (13737,))

In [0]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
latent_dim = 50
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


In [25]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 10
epochs = 50
print(val_samples)
val_samples//batch_size

13737


1373

In [31]:
import os
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf
if tf.test.gpu_device_name():
  print("GPU found")
else:
  print("No GPU found")


GPU found


In [32]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50


ResourceExhaustedError: ignored

In [0]:
model.save_weights('nms_weights.h5')

In [33]:
model.load_weights('nms_weights.h5')

OSError: ignored

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [0]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Simple Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Simple Translation:', decoded_sentence[:])